# Project 2


In [10]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  #To get figures with high quality!

import numpy as np
import torch
from torch import nn
from torch import optim
import matplotlib.pyplot as plt
import time
import torchvision

## Loading the dataset CIFAR10

We are told to classify birds from cats in the CIFAR10 dataset. This means that we will only use the images that are labeled as birds or cats. In the CIFAR10 these labels are 2 and 3 respectively. We will have the training, validation and test datasets.

In [11]:
from torchvision import datasets, transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

#Filtering the birds and cats images from the train set
trainset.targets = torch.tensor(trainset.targets)
mask = (trainset.targets == 2) | (trainset.targets == 3)
trainset.targets = (trainset.targets[mask] - 2)  # Adjust labels to be 0 for the birds and 1 cats
trainset.data = trainset.data[mask.numpy().astype(bool)]
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)


testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

#Filtering the birds and cats images from the test set
testset.targets = torch.tensor(testset.targets)
mask = (testset.targets == 2) | (testset.targets == 3)
testset.targets = (testset.targets[mask] - 2)  # Adjust labels to be 0 for the birds and 1 cats
testset.data = testset.data[mask.numpy().astype(bool)]

testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


Splitting the training into train and valid (80/20)

## Implementation of a Lenet5

In [12]:
class Lenet5(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_part = nn.Sequential( # As images are rgb we have 3 input channels
            nn.Conv2d(in_channels=3, out_channels=6,
                      kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120), # The input dimension at the classifier is 16 images of 5x5 so 16*5*5
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 2), # As we are just classifying two classes the output must be 2
            nn.LogSigmoid()

        )
        self.lr = 0.001 #Learning Rate

        self.optim = optim.Adam(self.parameters(), self.lr)

        self.criterion = nn.CrossEntropyLoss()

        self.loss_during_training = []

        self.valid_loss_during_training = []

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        if torch.cuda.is_available():
                print("Switching to GPU")
        else:
                print("GPU not available, running CPU")
        
        self.to(self.device)

    def forward(self, x):
        x = self.conv_part(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def trainloop(self,trainloader,validloader,epochs):

        self.train()
        for e in range(epochs):

            start_time = time.time()

            running_loss = 0.

            for images, labels in trainloader:

                # Move input and label tensors to the default device
                images, labels = images.to(self.device), labels.to(self.device)  
        
                self.optim.zero_grad()

                out = self.forward(images)


                loss = self.criterion(out,labels)

                running_loss += loss.item()


                loss.backward()
                self.optim.step()

            self.loss_during_training.append(running_loss/len(trainloader))

            with torch.no_grad():

                  self.eval()

                  running_loss = 0.

                  for images,labels in validloader:

                      # Move input and label tensors to the default device
                      images, labels = images.to(self.device), labels.to(self.device)  
        
                      out = self.forward(images)

                      #Loss function
                      loss = self.criterion(out,labels)

                      running_loss += loss.item()

                  self.valid_loss_during_training.append(running_loss/len(validloader))

                  self.train()


            print("Epoch %d. Training loss: %f, Validation loss: %f, Time per epoch: %f seconds" 
                      %(e,self.loss_during_training[-1],self.valid_loss_during_training[-1],
                       (time.time() - start_time)))
    
    def compute_accuracy(self, dataloader):
      correct = 0
      total = 0
      with torch.no_grad():
        self.eval()
        for images, labels in dataloader:
                # Move input and label tensors to the default device
                images, labels = images.to(self.device), labels.to(self.device)  
        
                outputs = self(images)

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
      return accuracy


In [13]:
model = Lenet5()
model.trainloop(trainloader,testloader,30)

Switching to GPU
Epoch 0. Training loss: 0.585952, Validation loss: 0.537629, Time per epoch: 3.761291 seconds
Epoch 1. Training loss: 0.511744, Validation loss: 0.516237, Time per epoch: 1.562532 seconds
Epoch 2. Training loss: 0.475808, Validation loss: 0.490622, Time per epoch: 1.520499 seconds
Epoch 3. Training loss: 0.459729, Validation loss: 0.472432, Time per epoch: 1.365472 seconds
Epoch 4. Training loss: 0.431958, Validation loss: 0.449626, Time per epoch: 1.367150 seconds
Epoch 5. Training loss: 0.414457, Validation loss: 0.441921, Time per epoch: 1.451922 seconds
Epoch 6. Training loss: 0.408774, Validation loss: 0.440843, Time per epoch: 1.381368 seconds
Epoch 7. Training loss: 0.398327, Validation loss: 0.440901, Time per epoch: 1.387728 seconds
Epoch 8. Training loss: 0.377847, Validation loss: 0.411737, Time per epoch: 1.415927 seconds
Epoch 9. Training loss: 0.361752, Validation loss: 0.416151, Time per epoch: 1.345981 seconds
Epoch 10. Training loss: 0.349519, Validati

In [14]:
#Tiempo con CPU: 
# sobre 1.8 segundos
# ACC is 0.79

#Tiempo con CPU: 
# sobre 1.4 segundos
# ACC is 0.8015, igual

In [15]:
print(model.compute_accuracy(testloader))

0.8015
